In [ ]:
import os, glob
import pathlib

import numpy as np
import skimage
import skimage.io as sio
import skimage.transform as strans

import matplotlib.pyplot as plt

import pandas as pd
import transforms3d

In [ ]:
# process CO3D data in data2 for a certain class
data_dir = '.'
type_f = 'teddybear' # hydrant, plant
idx_num = 1 # which idx

In [ ]:

viables = [_ for _ in sorted(os.listdir(os.path.join(data_dir,type_f))) if os.path.isdir(os.path.join(data_dir,type_f,_)) and len(os.path.join(data_dir,type_f,_).split('_')) == 3]
co3d_seq = viables[idx_num]
output_folder = type_f+'_'+co3d_seq
co3d_seq_folder = os.path.join(data_dir,type_f,co3d_seq)
co3d_seq,co3d_seq_folder

# Requirements

[unimatch](https://github.com/autonomousvision/unimatch) for generating optical flow


[XMem](https://github.com/hkchengrex/XMem) for propogating the first mask


In [ ]:
base_folder = co3d_seq_folder
input_folder = os.path.join(base_folder,'images')
output_folder = os.path.join('rvid','{}_{}'.format(type_f,co3d_seq))

target_size = 125000
gmflow_path = '../unimatch/'
xmem_path = '../XMem/'

frame1_mask = os.path.join(base_folder,'masks','frame000001.png')

imgs_folder = os.path.join(output_folder,'JPEGImages','video1')
silh_folder = os.path.join(output_folder,'Annotations','video1')
flow_folder = os.path.join(output_folder,'Flow','video1')

for gen_folder in [output_folder,imgs_folder,silh_folder,flow_folder]:
    if not os.path.exists(gen_folder):
        #import shutil
        #shutil.rmtree(gen_folder)
        os.makedirs(gen_folder)

In [ ]:
in_files = sorted(glob.glob(os.path.join(input_folder,'*.jpg')) + glob.glob(os.path.join(input_folder,'*.png')))

In [ ]:
PYo,PXo = sio.imread(in_files[0]).shape[:2]
init_scale = np.prod([PYo,PXo])
scales = {}
for i in range(10):
    scale = 2**i
    scales[scale] = init_scale/(scale**2)
scale_to_use = sorted([(abs(np.log(v/target_size)),k) for k,v in scales.items() ])[0][1]
PY,PX = int(round(PYo/scale_to_use)),int(round(PXo/scale_to_use))
scale_to_use,PY,PX

In [ ]:
valid_inputs = []
file_map = {}
for idx,file in enumerate(in_files):
    name = pathlib.Path(file).parts[-1]
    #if not os.path.exists(os.path.join(imgs_folder,name)):
    img = sio.imread(file)
    valid_inputs.append(img.sum() != 0)
    new_name = 'frame{:06d}.jpg'.format(sum(valid_inputs))
    if valid_inputs[-1] == False:
        continue
    #print(new_name)
    file_map[idx] = sum(valid_inputs)
    simg = strans.resize(img,(PY,PX))
    sio.imsave(os.path.join(imgs_folder,new_name),skimage.img_as_ubyte(simg))

In [ ]:
df = pd.read_json(os.path.join(*(base_folder.split('/')[:-1] + ['frame_annotations.jgz'])),compression={'method':'gzip'})
df2 = df[df.sequence_name == int(co3d_seq.replace('_',''))]
fls = []
pps = []
sizes = []
assert(len(df2) == len(valid_inputs))
for i,row in enumerate(df2.sort_values('frame_number').itertuples()):
    fn, imgd, maskd, view = row[2],row[4],row[6],row[7]
    if not valid_inputs[i]:
        continue
    fl = np.array(view['focal_length'])
    pp = np.array(view['principal_point'])
    sizeA = list(row[4]['size'])

    if 'intrinsics_format' in view and view['intrinsics_format'] == 'ndc_isotropic':
        half_image_size_wh_orig = np.array(list(reversed(sizeA))) / 2.0
        rescale = half_image_size_wh_orig.min()
        # principal point and focal length in pixels
        principal_point_px = half_image_size_wh_orig - pp * rescale
        focal_length_px = fl * rescale
    else:
        half_image_size_wh_orig = np.array(list(reversed(sizeA))) / 2.0
        # principal point and focal length in pixels
        principal_point_px = (
            -1.0 * (pp - 1.0) * half_image_size_wh_orig
        )
        focal_length_px = fl * half_image_size_wh_orig

    fls.append(focal_length_px)
    pps.append(principal_point_px)

    sizes.append(sizeA)
assert(np.array(sizes).std(0).sum() == 0) # same sizes
pp = np.array(pps).mean(0)
fl = np.array(fls).mean(0).mean()
meanpp = (np.array([pp[1],pp[0]])/np.array(sizes).mean(0)).mean() 
assert(abs(meanpp - 0.5) < 1e-3) # basically center of frame
fl = fl/scale_to_use

In [ ]:
len(valid_inputs),df2.shape,len(in_files)

In [ ]:
poses = []
depths = []
import skimage.io as sio
import skimage.transform as sktrans

for i,row in enumerate(df2.sort_values('frame_number').itertuples()):
    fn, imgd, maskd, view = row[2],row[4],row[6],row[7]
    depthd = row[5]
    if not valid_inputs[i]:
        continue
    maskd = maskd['path'][maskd['path'].index(co3d_seq):]
    imgd = imgd['path'][imgd['path'].index(co3d_seq):]
    
    Rmat = np.array(view['R'])
    Tvec = np.array(view['T'])
    Tvec = -Rmat @ Tvec
    q = transforms3d.quaternions.mat2quat(Rmat.T)
    poses.append(list(q) + list(Tvec))
    
    depth_r = sio.imread(os.path.join(data_dir,type_f,depthd['path'][depthd['path'].index(co3d_seq):]))#.astype(float)
    depth_m = sio.imread(os.path.join(data_dir,type_f,depthd['mask_path'][depthd['mask_path'].index(co3d_seq):])).astype(float)
    
    depth_r_s = depth_r.shape
    depth_r = depthd['scale_adjustment']*np.frombuffer(depth_r,dtype=np.float16).astype(np.float32).reshape(depth_r_s)

    valid_d = (depth_r > 0)
    depth_r[~valid_d] = np.nan
    depth_r = sktrans.resize(depth_r,(PY,PX),anti_aliasing=False,order=0)
    depths.append(depth_r)

In [ ]:
import pickle
import gzip
import trimesh
with gzip.open(os.path.join(output_folder,'pose_depth.pkl.gz'), "wb") as f:
    out_dict = {'fl':fl,'poses':poses,'depths':depths}
    ply_path = os.path.join(co3d_seq_folder,'pointcloud.ply')
    if os.path.exists(ply_path):
        out_dict['mesh'] = trimesh.load(ply_path)
    pickle.dump(out_dict, f)

In [ ]:
gm_flow_cmd = """CUDA_VISIBLE_DEVICES="0" python {} \
--inference_dir {} \
--output_path {} \
--pred_bidir_flow \
--save_flo_flow \
--resume {} {}
"""
alt_cmd =  '--inference_size {} {}'.format(PX*2,PY*2) if target_size < 1e5 else ''
gm_flow_cmd_f = gm_flow_cmd.format(os.path.join(gmflow_path,'main_flow.py'),imgs_folder,flow_folder,os.path.join(gmflow_path,'pretrained','gmflow-scale1-mixdata-train320x576-4c3a6e9a.pth'),alt_cmd)
print(gm_flow_cmd_f)

In [ ]:
if len(glob.glob(os.path.join(flow_folder,'*.flo'))) != (len(in_files)*2-2):
    os.system(gm_flow_cmd_f)

In [ ]:
imgM = sio.imread(frame1_mask)
simgM = skimage.img_as_ubyte(strans.resize(imgM,(PY,PX)) >0.5)
sio.imsave(os.path.join(silh_folder,pathlib.Path(in_files[0]).parts[-1].replace('.jpg','.png')),simgM)

In [ ]:
xmem_output = os.path.join(output_folder,'masks')
xmem_cmd = 'python {} --model {} --dataset G --generic_path {} --output {}'.format(os.path.join(xmem_path,'eval.py'),os.path.join(xmem_path,'saves','XMem.pth'),output_folder,xmem_output)

In [ ]:
os.system(xmem_cmd)

In [ ]:
xmem_cmd